## Generate 16 Variations of a single LUT Design

In [5]:
for i in range(1,16):
    output = f"{bin(i)[2:]}".zfill(16)
    # print(output)

    fh = open(f"./results/design_files/design_{i}.v","w+")
    # fh.write(f"{output}")
    fh.write(f"`timescale 1ns / 1ps\n")
    fh.write(f"\n")
    fh.write(f"module generic_func(lut_in3, lut_in2, lut_in1, lut_in0, lut_out);\n")
    fh.write(f"\n")
    # fh.write(f"input wire [3:0] lut_in;\n")
    for j in range(4):
        fh.write(f"input wire lut_in{j};\n")
    fh.write(f"output reg lut_out;\n")
    fh.write(f"\n")
    fh.write(f"always @(*) begin\n")
    fh.write(f"\tcase ({{lut_in3, lut_in2, lut_in1, lut_in0}})\n")
    for j in range(16):
        fh.write(f"\t\t4'b{bin(j)[2:].zfill(4)}: lut_out <= {output[j]};\n")
    fh.write(f"\t\tdefault: lut_out <= 0;\n")
    fh.write(f"\tendcase\n")
    fh.write(f"end\n")
    fh.write(f"endmodule\n")
    fh.close()

## OpenFPGA Build Commands
```
python3 openfpga_flow/scripts/run_fpga_task.py basic_tests/0_debug_task/bank_fpga
python3 openfpga_flow/scripts/run_fpga_task.py basic_tests/0_debug_task/frame_fpga
python3 openfpga_flow/scripts/run_fpga_task.py basic_tests/0_debug_task/cc_fpga
```

In [24]:
for i in range(16):
    print(f"bench{i}")

bench0
bench1
bench2
bench3
bench4
bench5
bench6
bench7
bench8
bench9
bench10
bench11
bench12
bench13
bench14
bench15


## Code to Generate 1,000 Randomized Designs

### Constraints 
- 2x2 CLB
- 16 LUTs (Random Number Instantiated)


In [4]:
import random
import os
import shutil

In [19]:
shutil.rmtree("./results/random_bitstreams")
os.mkdir(f"./results/random_bitstreams")

class Wire:
    def __init__(self,name,lut,idx,dir):
        self.name = name
        self.lut = lut
        self.idx = idx
        self.dir = dir

for i in range(1000):
    # output = f"{bin(i)[2:]}".zfill(16)
    # print(output)

    index = i + 1
    index_str = f"{index}".zfill(4)
    
    os.mkdir(f"./results/random_bitstreams/{index_str}")
    fh = open(f"./results/random_bitstreams/{index_str}/design.v","w+")

    # NUM_LUTS = random.randint(1,16)
    NUM_LUTS = random.randint(1,16)
    NUM_LUT_INPUTS = 4

    wires = []
   

    fh.write(f"// design #: {index_str}\n")
    fh.write(f"// num luts #: {NUM_LUTS}\n")
    fh.write(f"\n")
    fh.write(f"\n")


    fh.write(f"`timescale 1ns / 1ps\n")
    fh.write(f"\n")
    # fh.write(f"module generic_func(fpga_in_0, fpga_in_1, fpga_in_2, fpga_in_3")
    fh.write(f"module fpga_design(")
    for lut_input_idx in range(NUM_LUT_INPUTS):
        fh.write(f"fpga_in_{lut_input_idx}, ")
        # wires.append(Wire(f"fpga_in_{lut_input_idx}",-1,-1,"in"))
    for lut_idx in range(NUM_LUTS):
        if lut_idx < NUM_LUTS - 1:
            fh.write(f"fpga_out_{lut_idx}, ")
            # wires.append(Wire(f"fpga_out_{lut_idx}",-1,-1,"out"))
            
        else:
            fh.write(f"fpga_out_{lut_idx}")
    fh.write(f");\n")
    fh.write(f"\n")

    for j in range(NUM_LUT_INPUTS):
        fh.write(f"input wire fpga_in_{j};\n")

    for lut_idx in range(NUM_LUTS):
        fh.write(f"output reg fpga_out_{lut_idx};\n")


    # randomize connections between LUTs with wires
    for lut_idx in range(NUM_LUTS):
        fh.write(f"// LUT {lut_idx} Inputs \n")
        for lut_input_idx in range(NUM_LUT_INPUTS):
            in_wire_name = f"lut_{lut_idx}_in_{lut_input_idx}"
            fh.write(f"wire {in_wire_name};\n")
            wires.append(Wire(in_wire_name, lut_idx, lut_input_idx, "in"))
        out_wire_name = f"lut_{lut_idx}_out"
        fh.write(f"wire {out_wire_name};\n")

        ## temp
        fh.write(f"assign fpga_out_{lut_idx} = lut_{lut_idx}_out;\n")

        ## randomly assign lut inputs
        for lut_input_idx in range(NUM_LUT_INPUTS):
            lut_input_choice = random.randint(0,3)
            fh.write(f"assign lut_{lut_idx}_in_{lut_input_idx} = fpga_in_{lut_input_choice};\n")


        fh.write(f"\n")


    for lut_idx in range(NUM_LUTS):
        fh.write(f"\n")
        fh.write(f"always @(*) begin\n")
        fh.write(f"\tcase ({{")
        for lut_input_idx in range(NUM_LUT_INPUTS-1,-1,-1):
            if lut_input_idx > 0:
                fh.write(f"lut_{lut_idx}_in_{lut_input_idx}, ")
            else:
                fh.write(f"lut_{lut_idx}_in_{lut_input_idx}")
        fh.write(f"}})\n")
        for j in range(2**NUM_LUT_INPUTS):
            fh.write(f"\t\t4'b{bin(j)[2:].zfill(4)}: lut_{lut_idx}_out <= {random.randint(0,1)};\n")
        fh.write(f"\t\tdefault: lut_{lut_idx}_out <= 0;\n")
        fh.write(f"\tendcase\n")
        fh.write(f"end\n")

    fh.write(f"endmodule\n")
    fh.close()

In [20]:
# write task.conf

NUM_DESIGNS=1000

fh = open(f"./results/random_bitstreams/task.conf","w+")


fh.write(f"# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =\n")
fh.write(f"# Configuration file for running experiments\n")
fh.write(f"# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =\n")
fh.write(f"# timeout_each_job : FPGA Task script splits fpga flow into multiple jobs\n")
fh.write(f"# Each job execute fpga_flow script on combination of architecture & benchmark\n")
fh.write(f"# timeout_each_job is timeout for each job\n")
fh.write(f"# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =\n")
fh.write(f"\n")
fh.write(f"[GENERAL]\n")
fh.write(f"run_engine=openfpga_shell\n")
fh.write(f"power_tech_file = ${{PATH:OPENFPGA_PATH}}/openfpga_flow/tech/PTM_45nm/45nm.xml\n")
fh.write(f"power_analysis = true\n")
fh.write(f"spice_output=false\n")
fh.write(f"verilog_output=true\n")
fh.write(f"timeout_each_job = 20*60\n")
fh.write(f"fpga_flow=yosys_vpr\n")
fh.write(f"\n")
fh.write(f"[OpenFPGA_SHELL]\n")
fh.write(f"openfpga_shell_template=${{PATH:OPENFPGA_PATH}}/openfpga_flow/openfpga_shell_scripts/write_full_testbench_example_script.openfpga\n")
fh.write(f"openfpga_arch_file=${{PATH:OPENFPGA_PATH}}/openfpga_flow/openfpga_arch/k4_N4_40nm_cc_openfpga.xml\n")
fh.write(f"openfpga_sim_setting_file=${{PATH:OPENFPGA_PATH}}/openfpga_flow/openfpga_simulation_settings/auto_sim_openfpga.xml\n")
fh.write(f"openfpga_vpr_device_layout=\n")
fh.write(f"openfpga_fast_configuration=\n")
fh.write(f"\n")
fh.write(f"[ARCHITECTURES]\n")
fh.write(f"arch0=${{PATH:OPENFPGA_PATH}}/openfpga_flow/vpr_arch/k4_N4_tileable_40nm.xml\n")
fh.write(f"\n")
fh.write(f"[BENCHMARKS]")
for i in range(NUM_DESIGNS):
    design_num = f"{i}".zfill(4)
    fh.write(f"bench{i}=${{PATH:OPENFPGA_PATH}}/debug/architectures/arch_gen/results/random_bitstreams/{design_num}/design.v\n")
fh.write(f"\n")
fh.write(f"[SYNTHESIS_PARAM]\n")
fh.write(f"bench_read_verilog_options_common = -nolatches\n")
fh.write(f"\n")
for i in range(NUM_DESIGNS):
    design_num = f"{i}".zfill(4)
    fh.write(f"bench{i}_top=fpga_design\n")
fh.write(f"\n")
fh.write(f"[SCRIPT_PARAM_MIN_ROUTE_CHAN_WIDTH]\n")
fh.write(f"end_flow_with_test=\n")


fh.close()

In [ ]:
python3 openfpga_flow/scripts/run_fpga_task.py basic_tests/0_debug_task/random_designs --maxthreads=8 --debug